# 30-121 : Tidy up the Hyperparameter code

## Web References

- [Getting started with KerasTuner](https://keras.io/guides/keras_tuner/getting_started/)
- [Hyper parameter optimisation with Hyperopt and MLflow](https://www.youtube.com/watch?v=W_mJxQupJ4I)
- [hyperopt_logistic_regression.ipynb](https://github.com/amoat7/mlflow_tutorial/blob/master/notebooks/hyperopt_logistic_regression.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import logging
from functools import partial

import pandas as pd
import tensorflow as tf
import keras as k
import kerastuner as kt

from competition import source_data as sd
from competition import data_preparation as dp
from competition import feature_engineering as fe
from competition import model_data as md
from competition import model_training as mt
from competition import model_layers as ml
from competition import model_definitions as mm

import mlflow
import mlflow.keras

In [ ]:
# Set the GPU memory for growth
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Configure Logging

In [ ]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

## Data Collection

In [ ]:
# load the source training set
df_source = sd.read_csv('../data/train.csv.gz',
                        compression='gzip',
                        dtype=sd.source_dtype)

In [ ]:
# load the source training labels
df_source_labels = sd.read_csv('../data/train_labels.csv')

## Data Preparation & Cleaning

In [ ]:
# prepare the main dataset
df_source = dp.prepare_main_dataset(df_source,
                                    elapsed_time_min_clip=0,
                                    elapsed_time_max_clip=3691298)

# remove sessions with problems
problem_sessions = dp.find_problem_sessions(df_source)
df_source = df_source[~df_source['session_id'].isin(problem_sessions)]

In [ ]:
# prepare the label dataset
df_source_labels = dp.prepare_label_dataset(df_source_labels)

# remove sessions with problems
df_source_labels = df_source_labels[~df_source_labels['session_id'].isin(problem_sessions)]

## Feature Engineering

In [ ]:
# create the initial features
df_features = fe.create_initial_features(df_source, df_source_labels)

In [ ]:
# add the feature to the features dataset
df_features = fe.add_elapsed_time_features(
    features=df_features,
    X=df_source)

In [ ]:
# add the total count features to the features dataset
df_features = fe.add_count_total_features(
    features=df_features,
    X=df_source)

In [ ]:
# add the unique count features to the features dataset
df_features = fe.add_count_unique_features(
    features=df_features,
    X=df_source)

In [ ]:
# # add the screen heatmap feature to the features dataset
# df_features = fe.add_screen_heatmap_feature(df_features, df_source)

# with pd.option_context('display.max_columns', None):
#     display(df_features.head(6))

## Data Selection

In [ ]:
random_state = 51

In [ ]:

# split the dataset into train, validation and test sets
train, val, test = md.select_sessions(
    y=df_source_labels.head(100),
    random_state=random_state,
    test_size=0.60,
    train_size=0.75)

## Model Training

### Create Datasets

In [ ]:
# set the feature list
feature_list = ['elapsed_time_sum', 'elapsed_time_max', 'elapsed_time_min', 'elapsed_time_mean', 'elapsed_time_mode']

# create the simple model dataset
simple_model_dataset = md.get_feature_dataset(
    features=df_features,
    y=df_source_labels,
    feature_list=feature_list,
    train=train,
    val=val,
    test=test,
    include_question=True,
    expand_question=False)

In [ ]:
# define the simple model input shape
input_data = simple_model_dataset['train']['X']
simple_model_shape = input_data.shape[1]
print('simple_model_shape:', simple_model_shape)

# define the output shape
output_data = simple_model_dataset['train']['y']
simple_model_output_shape = 1
print('output_shape', simple_model_output_shape)

### Training

In [ ]:
# create the experiment
mlflow.set_experiment("Hyper Parameter Tuning")


#### Simple Dense

In [ ]:
#mlflow.end_run()

In [ ]:
# create the hyperparameter object
hp = kt.HyperParameters()

model = mm.tune_simple_dense_model(
    dataset=simple_model_dataset,
    max_trials=10,
    input_shape=simple_model_shape,
    output_shape=simple_model_output_shape,
    dense_layer_count=hp.Int('dense_layer_count', min_value=1, max_value=3, step=1),
    dense_units=hp.Int('dense_units', min_value=32, max_value=512, step=32),
    dense_activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid']),
    dense_l1_regulization=hp.Float('dense_l1_regulization', min_value=0.0, max_value=0.1, step=0.01),
    dense_l2_regulization=hp.Float('dense_l2_regulization', min_value=0.0, max_value=0.1, step=0.01),
    dense_dropout=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.05),
    train_epochs=10,
    train_batch_size=10,
    train_optimizer=k.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])),
    train_loss='binary_crossentropy',
    train_metrics=['accuracy'],
    train_class_weight=None)

model.summary()


In [ ]:
# define the hyperparameter tuning function
def build_simple_model(hp):
    global simple_model_shape
    global simple_model_output_shape

    # create the model
    model = mm.get_simple_dense_model(
        input_shape=simple_model_shape,
        output_shape=simple_model_output_shape,
        dense_layer_count=hp.Int('dense_layer_count', min_value=1, max_value=3, step=1),
        dense_units=hp.Int('dense_units', min_value=32, max_value=512, step=32),
        dense_activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid']),
        dense_l1_regulization=hp.Float('dense_l1_regulization', min_value=0.0, max_value=0.1, step=0.01),
        dense_l2_regulization=hp.Float('dense_l2_regulization', min_value=0.0, max_value=0.1, step=0.01),
        dense_dropout=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.05),
        compile=True,
        optimizer=k.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])

    return model

# define CustomSearch class
class CustomSearch(kt.tuners.RandomSearch):
    # def pre_create_trial(self):
    #     logging.info('pre_create_trial')
    #     mlflow.start_run(nested=True)
    #     super(CustomSearch, self).pre_create_trial()

    def on_trial_begin(self, trial):
        logging.info('on_trial_begin')
        #if int(trial.trial_id) > 0:
        mlflow.keras.autolog()
        mlflow.start_run(nested=True)
            #mlflow.keras.autolog(disable=False)

        super(CustomSearch, self).on_trial_begin(trial)

    def on_trial_end(self, trial):
        logging.info('on_trial_end')
        mlflow.end_run()

        super(CustomSearch, self).on_trial_end(trial)

# create the tuner
with mlflow.start_run() as run:
    tuner = CustomSearch(
        build_simple_model,
        objective="val_accuracy",
        max_trials=10,
        executions_per_trial=1,
        overwrite=True)
    
    run_id = run.info.run_id
mlflow.end_run()
mlflow.delete_run(run_id)

with mlflow.start_run():
    mlflow.keras.autolog()
    #mlflow.start_run(nested=True)

    # search for the best hyperparameters
    tuner.search(
        simple_model_dataset['train']['X'],
        simple_model_dataset['train']['y'],
        validation_data=(simple_model_dataset['val']['X'], simple_model_dataset['val']['y']),
        epochs=10,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])
    
    #mlflow.end_run()

    # log the best hyperparameters
    best_hp = tuner.get_best_hyperparameters()[0].values
    mlflow.log_params(best_hp)
    print(best_hp)

    # Retrieve the best model, evaluate it, and log the metrics
    best_model = tuner.get_best_models()[0]
    val_loss, val_accuracy = best_model.evaluate(simple_model_dataset['val']['X'], simple_model_dataset['val']['y'])
    mlflow.log_metric("val_loss", val_loss)
    mlflow.log_metric("val_accuracy", val_accuracy)

mlflow.end_run()

In [ ]:
# # log the parameters
# mlflow.set_tag('model', 'simple-dense-model')
# mlflow.set_tag('notebook', '03-101_mlflow.ipynb')
# mt.log_params(simple_model_dataset, feature_list, random_state)

# # train the model
# model = mm.train_simple_dense(
#     dataset=simple_model_dataset,
#     input_shape=simple_model_shape,
#     output_shape=simple_model_output_shape,
#     dense_layer_count=2,
#     dense_units=1024,
#     dense_activation='relu',
#     dense_l1_regulization=1e-5,
#     dense_l2_regulization=0.0,
#     dense_dropout=0.0,
#     train_epochs=1000,
#     train_batch_size=1000,
#     train_optimizer='adam',
#     train_loss='binary_crossentropy',
#     train_metrics=['accuracy'],
#     train_class_weight=None)

# mlflow.end_run()